In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import sys
sys.path.append('../YOLOv3')

In [ ]:
from models import *
from utils.logger import *
from utils.utils import *
from utils.datasets_copy import *
from utils.parse_config import *
from val import evaluate
import tqdm

In [ ]:
from terminaltables import AsciiTable

In [ ]:
import os
import sys
import time
import datetime
import argparse
import json

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import pandas as pd

In [ ]:
class Config:
    
    def __init__(self):
        self.epochs = 4
        self.batch_size = 8
        self.gradient_accumulations = 2
        self.model_def = '<base_path>/iWildCam2020/YOLOv3/config/yolov3.cfg'
        self.data_config = "<base_path>/iWildCam2020/YOLOv3/config/iWildCam_test.data"
        self.pretrained_weights = 'path_to_checkpoint.pth'
        self.n_cpu = 2
        self.img_size = 512
        self.checkpoint_interval = 1
        self.evaluation_interval = 1
        self.compute_map = False
        self.multiscale_training = False
        self.lr = 1e-4
        
        self.log_dir = '<base_path>/iWildCam2020/YOLOv3/logs'
        self.output =  '<base_path>/iWildCam2020/YOLOv3/output'
        self.checkpoints =  '<base_path>/iWildCam2020/YOLOv3/checkpoints'
        
        self.device = 'cuda:0'
        self.multigpu = False
        
opt = Config()

In [ ]:
logger = Logger(opt.log_dir)

In [ ]:
os.makedirs(opt.output, exist_ok=True)
os.makedirs(opt.checkpoints, exist_ok=True)

In [ ]:
# Get data configuration
data_config = parse_data_config(opt.data_config)
train_path = data_config["train"]
valid_path = data_config["valid"]
class_names = load_classes(data_config["names"])

In [ ]:
# Initiate model
model = Darknet(opt.model_def)
model.apply(weights_init_normal)
# If specified we start from checkpoint
if opt.pretrained_weights:
    if opt.pretrained_weights.endswith(".pth"):
        model.load_state_dict(torch.load(opt.pretrained_weights))
    else:
        model.load_darknet_weights(opt.pretrained_weights)
if opt.multigpu:
    model = nn.DataParallel(model)
model = model.to(opt.device)
model = model.eval()

In [ ]:
# Get dataloader
dataset = ListDataset(valid_path, augment=False, multiscale=opt.multiscale_training, mode='test')
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=opt.batch_size,
    shuffle=False,
    num_workers=opt.n_cpu,
    pin_memory=True,
    collate_fn=dataset.collate_fn,
)

In [ ]:
root = '<base_path>/iWildCam2020/data/'
class_idx_path = os.path.join(root, 'category_idx_map.json')
with open(class_idx_path, 'r') as f:
    class_idx_map = json.loads(f.read())
idx_class_map = {int(v): int(k) for k, v in class_idx_map.items()}

In [ ]:
predicted_classes = []
images = []

In [ ]:
start = time.time()
for batch_i, (paths, imgs, targets, cls, is_empty) in enumerate(tqdm.tqdm(dataloader, desc='Testing')):
    with torch.no_grad():
        batches_done = batch_i

        imgs = Variable(imgs.to(opt.device))
        targets = Variable(targets.to(opt.device), requires_grad=False)
        cls = Variable(cls.to(opt.device), requires_grad=False)

        outputs, class_scores = model(imgs)
        pred_class = torch.argmax(class_scores, dim=1).numpy().tolist()
        predicted_classes += [m for m in map(lambda x: idx_class_map[x], pred_class)]
        images += [path.split(os.path.sep)[-1].split('.')[0] for path in paths]

end = time.time()
print('Time Required: {}'.format(end - start))

In [ ]:
df = pd.DataFrame({'Id': images, 'Category': predicted_classes})

In [ ]:
df.to_csv(os.path.join(root, 'test_data', 'test_predictions_cpt4.csv'), index=None)

In [ ]:
df.shape